# Advanced Water Rocket Optimization Examples

This notebook demonstrates advanced usage of the water rocket optimizer including:
- Custom parameter bounds
- Different optimization targets (altitude, velocity, flight time)
- Custom optimizer settings
- Comparison of optimization strategies

In [1]:
import time
import numpy as np
from waterrocketpy.optimization.water_rocket_optimizer import (
    WaterRocketOptimizer,
    optimize_for_altitude,
    optimize_for_velocity,
    optimize_for_flight_time,
)

## Helper Function for Results Display

In [2]:
def print_results(result, test_name):
    """Print optimization results in a nice format."""
    print(f"\n{'='*60}")
    print(f"RESULTS FOR: {test_name}")
    print(f"{'='*60}")
    print(f"Success: {result['success']}")
    print(f"Target: {result['target']}")
    print(f"Best value: {result['best_value']:.4f}")
    print(f"Evaluations: {result['n_evaluations']}")

    print("\nOptimal Parameters:")
    for param, value in result["best_params"].items():
        if param == "p_max_bar":
            print(f"  {param:20}: {value:8.2f} bar")
        elif param in ["L_body", "d_body"]:
            print(f"  {param:20}: {value:8.4f} m ({value*100:6.2f} cm)")
        elif param == "nozzle_diameter":
            print(f"  {param:20}: {value:8.4f} m ({value*1000:6.2f} mm)")
        elif param == "water_fraction":
            print(f"  {param:20}: {value:8.4f} ({value*100:6.2f}%)")
        else:
            print(f"  {param:20}: {value:8.4f}")

    if result["best_flight_data"] is not None:
        fd = result["best_flight_data"]
        print("\nFlight Performance:")
        print(f"  Max Altitude        : {fd.max_altitude:8.4f} m")
        print(f"  Max Velocity        : {fd.max_velocity:8.4f} m/s")
        print(f"  Flight Time         : {fd.flight_time:8.4f} s")
        print(f"  Water Depletion Time: {fd.water_depletion_time:8.4f} s")

## 1. Custom Bounds for Velocity Optimization

Define custom parameter bounds around typical bottle rocket dimensions and optimize for maximum velocity.

In [3]:
print("Velocity optimization with custom bounds")

# Define tighter bounds around typical values
custom_bounds = [
    (0.20, 0.30),    # L_body: 20-30 cm
    (0.085, 0.095),  # d_body: 8.5-9.5 cm (around 2L bottle)
    (7.0, 9.0),      # p_max_bar: 7-9 bar
    (0.009, 0.012),  # nozzle_diameter: 9-12 mm
    (0.25, 0.40),    # water_fraction: 25-40%
]

print("Custom bounds:")
param_names = ['L_body (m)', 'd_body (m)', 'p_max (bar)', 'nozzle_dia (m)', 'water_frac']
for i, (param_name, (min_val, max_val)) in enumerate(zip(param_names, custom_bounds)):
    print(f"  {param_name:15}: [{min_val:6.3f}, {max_val:6.3f}]")

start_time = time.time()

result_velocity = optimize_for_velocity(
    bounds=custom_bounds,
    method="differential_evolution",
    maxiter=25,
    popsize=10,
    seed=123,
)

elapsed_time = time.time() - start_time
print(f"\nOptimization completed in {elapsed_time:.2f} seconds")

print_results(result_velocity, "Custom Bounds Velocity Optimization")

Velocity optimization with custom bounds
Custom bounds:
  L_body (m)     : [ 0.200,  0.300]
  d_body (m)     : [ 0.085,  0.095]
  p_max (bar)    : [ 7.000,  9.000]
  nozzle_dia (m) : [ 0.009,  0.012]
  water_frac     : [ 0.250,  0.400]
Starting optimization for max_velocity using differential_evolution
Parameter bounds: [(0.2, 0.3), (0.085, 0.095), (7.0, 9.0), (0.009, 0.012), (0.25, 0.4)]
New best max_velocity: 43.1361 at evaluation 1
  Params: L_body=0.220, d_body=0.088, p_max=7.2bar, nozzle_d=0.0109, water_frac=0.380
New best max_velocity: 43.8598 at evaluation 2
  Params: L_body=0.205, d_body=0.089, p_max=7.5bar, nozzle_d=0.0108, water_frac=0.285
New best max_velocity: 44.9703 at evaluation 3
  Params: L_body=0.223, d_body=0.093, p_max=8.8bar, nozzle_d=0.0091, water_frac=0.399
New best max_velocity: 45.1061 at evaluation 4
  Params: L_body=0.247, d_body=0.090, p_max=8.0bar, nozzle_d=0.0101, water_frac=0.385
New best max_velocity: 47.0032 at evaluation 5
  Params: L_body=0.290, d_bod

## 2. Advanced Custom Optimizer Settings

Use the `WaterRocketOptimizer` class directly for more control over simulation settings and optimization parameters.

In [4]:
print("Advanced optimization with custom settings")

# Create optimizer with custom simulation settings
optimizer = WaterRocketOptimizer(
    L_cone=0.08,  # Fixed nose cone length
    material_name="PET",  # Fixed material
    simulation_settings={
        "max_time": 40,    # Longer simulation time
        "time_step": 0.01, # High precision
        "solver": "RK45",
    },
)

# Use default bounds but modify pressure range
bounds = optimizer.get_default_bounds()
bounds[2] = (5.0, 11.0)  # Wider pressure range: 5-11 bar

print("Modified bounds (pressure range widened):")
bound_names = ['L_body', 'd_body', 'p_max_bar', 'nozzle_diameter', 'water_fraction']
for name, (min_val, max_val) in zip(bound_names, bounds):
    print(f"  {name:15}: [{min_val:6.3f}, {max_val:6.3f}]")

start_time = time.time()

result_advanced = optimizer.optimize(
    bounds=bounds,
    target="max_altitude",
    method="differential_evolution",
    maxiter=30,
    popsize=12,
    atol=1e-6,  # Higher accuracy
    seed=456,
)

elapsed_time = time.time() - start_time
print(f"\nOptimization completed in {elapsed_time:.2f} seconds")

print_results(result_advanced, "Advanced Custom Settings")

Advanced optimization with custom settings
Modified bounds (pressure range widened):
  L_body         : [ 0.100,  0.500]
  d_body         : [ 0.050,  0.120]
  p_max_bar      : [ 5.000, 11.000]
  nozzle_diameter: [ 0.005,  0.025]
  water_fraction : [ 0.100,  0.800]
Starting optimization for max_altitude using differential_evolution
Parameter bounds: [(0.1, 0.5), (0.05, 0.12), (5.0, 11.0), (0.005, 0.025), (0.1, 0.8)]
New best max_altitude: 88.5795 at evaluation 1
  Params: L_body=0.448, d_body=0.105, p_max=8.0bar, nozzle_d=0.0063, water_frac=0.309
New best max_altitude: 105.1120 at evaluation 4
  Params: L_body=0.412, d_body=0.120, p_max=7.7bar, nozzle_d=0.0235, water_frac=0.284
New best max_altitude: 119.1762 at evaluation 6
  Params: L_body=0.395, d_body=0.106, p_max=10.0bar, nozzle_d=0.0188, water_frac=0.288
New best max_altitude: 120.2927 at evaluation 31
  Params: L_body=0.405, d_body=0.114, p_max=10.6bar, nozzle_d=0.0130, water_frac=0.226
New best max_altitude: 126.0110 at evaluati

## 3. Flight Time Optimization with Minimize Method

Use the `minimize` method with a good initial guess for gradient-based optimization.

In [5]:
print("Flight time optimization using minimize method")

# Starting point based on reasonable rocket dimensions
initial_guess = [0.25, 0.088, 8.0, 0.01, 0.3]

print("Initial guess:")
guess_names = ['L_body (m)', 'd_body (m)', 'p_max (bar)', 'nozzle_dia (m)', 'water_frac']
for name, value in zip(guess_names, initial_guess):
    print(f"  {name:15}: {value:8.4f}")

start_time = time.time()

result_minimize = optimize_for_flight_time(
    method="minimize",
    x0=initial_guess,
    options={"maxiter": 50, "disp": True},
)

elapsed_time = time.time() - start_time
print(f"\nOptimization completed in {elapsed_time:.2f} seconds")

print_results(result_minimize, "Flight Time Optimization (Minimize)")

Flight time optimization using minimize method
Initial guess:
  L_body (m)     :   0.2500
  d_body (m)     :   0.0880
  p_max (bar)    :   8.0000
  nozzle_dia (m) :   0.0100
  water_frac     :   0.3000
Starting optimization for flight_time using minimize
Parameter bounds: [(0.1, 0.5), (0.05, 0.12), (2.0, 12.0), (0.005, 0.025), (0.1, 0.8)]
New best flight_time: 8.3092 at evaluation 1
  Params: L_body=0.250, d_body=0.088, p_max=8.0bar, nozzle_d=0.0100, water_frac=0.300
New best flight_time: 8.3092 at evaluation 2
  Params: L_body=0.250, d_body=0.088, p_max=8.0bar, nozzle_d=0.0100, water_frac=0.300
New best flight_time: 8.3092 at evaluation 3
  Params: L_body=0.250, d_body=0.088, p_max=8.0bar, nozzle_d=0.0100, water_frac=0.300
New best flight_time: 20.0000 at evaluation 7
  Params: L_body=0.500, d_body=0.120, p_max=8.0bar, nozzle_d=0.0050, water_frac=0.800

Optimization completed!
Best flight_time: 20.0000
Best parameters:
  L_body: 0.5000
  d_body: 0.1200
  p_max_bar: 7.9980
  nozzle_dia

## 4. Comparison of Different Optimization Targets

Compare optimization results for altitude, velocity, and flight time using identical bounds and settings.

In [6]:
print("Comparing different optimization targets")

# Use the same bounds and settings for fair comparison
common_bounds = [
    (0.15, 0.35),   # L_body: 15-35 cm
    (0.07, 0.10),   # d_body: 7-10 cm
    (6.0, 10.0),    # p_max_bar: 6-10 bar
    (0.008, 0.015), # nozzle_diameter: 8-15 mm
    (0.25, 0.45),   # water_fraction: 25-45%
]

common_settings = {
    "method": "differential_evolution",
    "maxiter": 20,
    "popsize": 8,
    "seed": 789,
}

print("Common bounds for all targets:")
for i, (name, (min_val, max_val)) in enumerate(zip(param_names, common_bounds)):
    print(f"  {name:15}: [{min_val:6.3f}, {max_val:6.3f}]")

results = {}

print("\nOptimizing for altitude...")
results["altitude"] = optimize_for_altitude(
    bounds=common_bounds, **common_settings
)

print("Optimizing for velocity...")
results["velocity"] = optimize_for_velocity(
    bounds=common_bounds, **common_settings
)

print("Optimizing for flight time...")
results["flight_time"] = optimize_for_flight_time(
    bounds=common_bounds, **common_settings
)

Comparing different optimization targets
Common bounds for all targets:
  L_body (m)     : [ 0.150,  0.350]
  d_body (m)     : [ 0.070,  0.100]
  p_max (bar)    : [ 6.000, 10.000]
  nozzle_dia (m) : [ 0.008,  0.015]
  water_frac     : [ 0.250,  0.450]

Optimizing for altitude...
Starting optimization for max_altitude using differential_evolution
Parameter bounds: [(0.15, 0.35), (0.07, 0.1), (6.0, 10.0), (0.008, 0.015), (0.25, 0.45)]
New best max_altitude: 80.6665 at evaluation 1
  Params: L_body=0.305, d_body=0.087, p_max=7.7bar, nozzle_d=0.0114, water_frac=0.438
New best max_altitude: 91.8359 at evaluation 6
  Params: L_body=0.260, d_body=0.092, p_max=9.0bar, nozzle_d=0.0137, water_frac=0.300
New best max_altitude: 95.0675 at evaluation 10
  Params: L_body=0.322, d_body=0.088, p_max=9.2bar, nozzle_d=0.0116, water_frac=0.434
New best max_altitude: 98.9362 at evaluation 13
  Params: L_body=0.292, d_body=0.100, p_max=9.2bar, nozzle_d=0.0146, water_frac=0.267
New best max_altitude: 100.80

### Comparison Table

In [7]:
# Print comparison table
print(f"\n{'='*80}")
print("COMPARISON OF DIFFERENT TARGETS")
print(f"{'='*80}")

print(
    f"{'Target':<12} {'Best Value':<12} {'L_body(cm)':<10} {'d_body(cm)':<10} "
    f"{'P(bar)':<8} {'Nozzle(mm)':<10} {'Water%':<8}"
)
print("-" * 80)

for target, result in results.items():
    params = result["best_params"]
    print(
        f"{target:<12} {result['best_value']:<12.4f} "
        f"{params['L_body']*100:<10.2f} {params['d_body']*100:<10.2f} "
        f"{params['p_max_bar']:<8.2f} {params['nozzle_diameter']*1000:<10.2f} "
        f"{params['water_fraction']*100:<8.1f}"
    )


COMPARISON OF DIFFERENT TARGETS
Target       Best Value   L_body(cm) d_body(cm) P(bar)   Nozzle(mm) Water%  
--------------------------------------------------------------------------------
altitude     113.1028     35.00      10.00      10.00    15.00      32.0    
velocity     54.6777      35.00      7.60       10.00    15.00      32.2    
flight_time  9.7498       34.87      9.79       10.00    8.66       34.4    


## 5. Performance Analysis

Analyze how each optimized configuration performs across all metrics.

In [8]:
print(f"\n{'='*80}")
print("CROSS-PERFORMANCE ANALYSIS")
print(f"{'='*80}")
print(
    f"{'Optimized For':<15} {'Altitude(m)':<12} {'Velocity(m/s)':<13} {'Flight Time(s)':<14}"
)
print("-" * 80)

for target, result in results.items():
    if result["best_flight_data"] is not None:
        fd = result["best_flight_data"]
        print(
            f"{target:<15} {fd.max_altitude:<12.4f} {fd.max_velocity:<13.4f} {fd.flight_time:<14.4f}"
        )

print("\n📊 Key Observations:")
print("• Different targets lead to different optimal configurations")
print("• There are trade-offs between altitude, velocity, and flight time")
print("• Custom bounds help focus the search on realistic parameter ranges")
print("• The minimize method can be faster for gradient-based optimization")


CROSS-PERFORMANCE ANALYSIS
Optimized For   Altitude(m)  Velocity(m/s) Flight Time(s)
--------------------------------------------------------------------------------
altitude        113.1028     54.2000       9.7232        
velocity        105.6082     54.6777       9.3605        
flight_time     108.3814     47.9386       9.7498        

📊 Key Observations:
• Different targets lead to different optimal configurations
• There are trade-offs between altitude, velocity, and flight time
• Custom bounds help focus the search on realistic parameter ranges
• The minimize method can be faster for gradient-based optimization


## Summary

This notebook demonstrated advanced optimization techniques:

1. **Custom Bounds**: Restrict parameter ranges to realistic values
2. **Custom Settings**: Use the `WaterRocketOptimizer` class for fine control
3. **Different Methods**: Compare `differential_evolution` vs `minimize`
4. **Multi-Objective Analysis**: Compare different optimization targets
5. **Performance Trade-offs**: Understand the relationships between parameters

### Next Steps
- Experiment with different bound ranges
- Try different optimization algorithms
- Add constraints for specific design requirements
- Implement multi-objective optimization